## 1025. Divisor Game

Straightfoward method: dynamic programming. For each num <= N, if any divisor of num x satifies that Alice would fail for (num-x), Alice must be able to win for num.  

In [3]:
class Solution:
    def divisorGame(self, N: int) -> bool:
        arr = [0] * (N + 1)
        for num in range(2, N+1):
            win = False
            for i in range(1, num):
                if num % i == 0 and arr[num-i] == 0:
                    win = True
                    break
            arr[num] = 1 if win else 0
        return arr[N] == 1

With some simple analysis, we find that Alice could win iff N is even.
If N is odd, after any move, the next N would be even (since all divisors of an odd number are odd). Given some initial values:

1 -> False

2 -> True

3 -> False

4 -> True

It is observed that if Alice gets an odd number, she will have to turn that into even number, which leads to Bos's winning.

And, after Alice lose once at N, she would definately win for (N + 1), because she could make her first move by reducing N by one.

Thanks to https://leetcode.com/problems/divisor-game/discuss/274566/just-return-N-2-0-(proof)

In [ ]:
class Solution:
    def divisorGame(self, N: int) -> bool:
        return True if N % 2 == 0 else False

## 1026. Maximum Difference Between Node and Ancestor

In [ ]:
class Solution:
    def maxAncestorDiff(self, root: TreeNode) -> int:
        path_so_far = []
        max_so_far = [0]
        self.dfs(root, path_so_far, max_so_far)
        return max_so_far[0]
        
    def dfs(self, node, path_so_far, max_so_far):
        path_so_far.append(node.val)
        if node.left is None and node.right is None:
            max_so_far[0] = max(max_so_far[0], max(path_so_far) - min(path_so_far))
            path_so_far.pop()
            return
        if node.left is not None:
            self.dfs(node.left, path_so_far, max_so_far)
        if node.right is not None:
            self.dfs(node.right, path_so_far, max_so_far)
        path_so_far.pop()
        return

## 1027. Longest Arithmetic Sequence

__WRONG__ answer: we start by looking for the longest subsequence starting with each element. After finding any arithmetic sequence, we calculate the next expected value, and store this infomation in a dictionary as:

{expected_value: (diff, cur_length)}

The problem about this approach is that any expected_value might be expected in different sequences, we have to store all the possible (diff, cur_length) pairs for any expected_value.

So my solution will fail for case like [1, 2, 3, 4, 5, 7, 9, 11, 13, 15, 17, 19]. Because when it first met the number 5, sequence [1, 2, 3, 4, 5] is longer than [1, 3, 5], so it just discard the sequence with a diff of 2.

In [ ]:
# This is a WRONG answer!! (Greedy algo)
class Solution:
    def longestArithSeqLength(self, A: List[int]) -> int:
        max_len = 0
        for idx1 in range(len(A)):
            val_map = dict()
            for idx2 in range(idx1 + 1, len(A)):
                if A[idx2] in val_map:
                    diff = val_map[A[idx2]][0]
                    expected = A[idx2] + diff
                    max_so_far = val_map[A[idx2]][1] + 1
                    if expected not in val_map or val_map[expected][1] < max_so_far:
                        val_map[expected] = (diff, max_so_far)
                else:
                    diff = A[idx2] - A[idx1]
                    expected = A[idx2] + diff
                    max_so_far = 2
                    if expected not in val_map or val_map[expected][1] < max_so_far:
                        val_map[expected] = (diff, max_so_far)
            if len(val_map) > 0:
                max_len = max(max_len, max([x[1] for x in val_map.values()]))
        return max_len

Correct answer with DP. To start with, we try to design a DP function like this:

```
dp(m) = the length of longest arithmetic subsequences ending at m.
```
But it's difficult to come up with any updating methods, since we haven't taken into consideration the common difference. In this case, a good approach is to use 2D DP instead.
```
dp(m, k) = the length of longest arithmetic subsequence ending at m with common difference k.
```
Now clearly dp(m, k) = dp(m-k, k) + 1

In [13]:
# Accepted DP solution.
class Solution:
    def longestArithSeqLength(self, A: List[int]) -> int:
        matrix = [{} for _ in range(len(A))]
        max_so_far = 0
        for idx1 in range(1, len(A)):
            for idx2 in range(idx1):
                diff = A[idx1] - A[idx2]
                if diff in matrix[idx2]:
                    matrix[idx1][diff] = matrix[idx2][diff] + 1
                else:
                    matrix[idx1][diff] = 2
            max_so_far = max(max_so_far, max(matrix[idx1].values()))
        return max_so_far

[[1], [1], [1]]